In [1]:
import sys
sys.path.append('/Users/bhargavdesai/Desktop/IU/RA/CNS/hra-amap-main/hra-amap/code')

import json
import uuid
import random
import trimesh

from tissue import TissueBlock
from dataclass import Projection
from rui import RUIProcessor

from copy import deepcopy
from pathlib import Path
from tqdm.auto import tqdm
from datetime import datetime

In [2]:
# set seed for reproducible UUIDs
rd = random.Random()
rd.seed(12)


##### Load `TissueBlock`(s)

In [3]:
# define the target
target_name = 'VHFPancreas'

# load the millitome
millitome = trimesh.load('../data/Pancreas/Source/generic-pancreas-organ.glb')

##### Create virtual `Donors` and respective `RUI Locations`

In [4]:
tissue_blocks = []

for label, block in millitome.geometry.items():
    # create virtual donor data (since this is not available for Millitome objects YET)
    donor = {'sex': 'Female', 
             'label': label,
             'provider_name': 'MC-IU', 
             'provider_uuid': str(uuid.UUID(int=rd.getrandbits(128), version=4)), 
             'consortium_name': 'HRA',
             'id': 'https://purl.humanatlas.io/millitome/generic-pancreas-female',      # make necessary edits here
             'link': 'https://purl.humanatlas.io/millitome/generic-pancreas-female'}        # make necessary edits here

    # create the rui_location data
    rui_location = {'@context': "https://hubmapconsortium.github.io/ccf-ontology/ccf-context.jsonld",
                    '@id': f"{donor['id']}#{label}",
                    '@type': 'SpatialEntity',
                    'creator': 'Bhargav Snehal Desai',      # make necessary edits here
                    'creator_first_name': 'Bhargav Snehal',     # make necessary edits here
                    'creator_last_name': 'Desai',       # make necessary edits here
                    'creator_orcid': 'https://orcid.org/0009-0008-6509-7698',       # make necessary edits here
                    'label': label,
                    'creation_date': datetime.today().strftime('%Y-%m-%d'), 
                    'dimension_units': 'millimeter', 
                    'placement': {'@context': "https://hubmapconsortium.github.io/ccf-ontology/ccf-context.jsonld",
                                   '@id': f"{donor['id']}#{label}_placement", 
                                   '@type': 'SpatialPlacement', 
                                   'target': f'http://purl.org/ccf/latest/ccf.owl#{target_name}', 
                                   'placement_date': datetime.today().strftime('%Y-%m-%d'), 
                                   'scaling_units': 'ratio', 
                                   'rotation_order': 'XYZ', 
                                   'rotation_units': 'degree', 
                                   'translation_units': 'millimeter'
                                   }
    }
    
    # create TissueBlock objects
    tissue_block = TissueBlock.from_millitome(block, donor=donor, metadata=rui_location, target_name=target_name, label=label)
    tissue_blocks.append(tissue_block)

# assign random colors to each tissue block (for visualization)
for tissue_block in tissue_blocks:
    tissue_block.visual.vertex_colors = trimesh.visual.random_color()

##### Project

In [5]:
# load saved projections (check Usage.ipynb or Bidirectional Projections.ipynb on how to create projections)
projections = Projection.load('../results/Projections/Forward/GenericPancreas_to_VHF_Pancreas-97a29b28-9e3d-41fe-8027-f003f396a5ce/projections.pickle')

In [6]:
# project
projected_blocks = [projections.project(block) for block in tissue_blocks]

# create an axis aligned bounding boxes around the projected blocks
projected_blocks_aabb = [block.bounding_box for block in deepcopy(projected_blocks)]

# ensure same colors on the axis-aligned bounding boxes
for index, aabb in enumerate(projected_blocks_aabb):
    aabb.visual.vertex_colors = projected_blocks[index].visual.vertex_colors[0]

##### Create `dataset-graph.jsonld` and `extraction-sites.jsonld`

In [7]:
# path where registration jsonlds will be saved
registrations_dir = '../data/Millitome/Generic Pancreas'

In [8]:
# initialize processor
processor = RUIProcessor(blocks=projected_blocks, registration_dir=registrations_dir)

# initialize registration
processor.initialize_registration()

# generate the rui-locations.jsonld
processor.generate_rui_locations()

In [9]:
# load the file
with open(Path(registrations_dir).joinpath('rui_locations.jsonld')) as f:
  jsonld = json.load(f)

# extract the rui_locations
extraction_sites = list()
for donor in jsonld['@graph']:
  for sample in tqdm(donor['samples']):
    extraction_sites.append(sample['rui_location'])

# create the dataset-graph.jsonld
with open(Path(registrations_dir).joinpath('dataset-graph.jsonld'), 'w') as f:
  json.dump(jsonld, f, indent=2)

# create the extraction-sites.jsonld
with open(Path(registrations_dir).joinpath('extraction-sites.jsonld'), 'w') as f:
  json.dump(extraction_sites, f, indent=2)

  0%|          | 0/97 [00:00<?, ?it/s]

##### Visualization

In [10]:
# create scene with the projected tissue blocks
trimesh.Scene([projected_blocks]).show()

In [11]:
# create scene with the projected tissue blocks (EUI view)
trimesh.Scene([projected_blocks_aabb]).show()